In [2]:
import requests,pprint,json,datetime,time
import pymongo
from pymongo import MongoClient
#from secrets import *
import secrets

In [3]:
client = MongoClient('localhost')

<h2>Acquiring data from an API

In [4]:
city='London'
cidyId='2643743'
a = secrets.key

In [5]:
#requestString='http://api.openweathermap.org/data/2.5/forecast?id=524901&APPID=%s' % (a)
requestString=u'http://api.openweathermap.org/data/2.5/weather?q=%s&APPID=%s' % (city,a)

In [6]:
res=requests.get(requestString)

In [7]:
res.reason

'OK'

In [8]:
res.status_code

200

In [9]:
pprint.pprint(res.json())

{'base': 'stations',
 'clouds': {'all': 100},
 'cod': 200,
 'coord': {'lat': 51.51, 'lon': -0.13},
 'dt': 1574183889,
 'id': 2643743,
 'main': {'humidity': 81,
          'pressure': 1016,
          'temp': 277.67,
          'temp_max': 279.82,
          'temp_min': 275.37},
 'name': 'London',
 'rain': {},
 'sys': {'country': 'GB',
         'id': 1414,
         'sunrise': 1574148288,
         'sunset': 1574179629,
         'type': 1},
 'timezone': 0,
 'visibility': 10000,
 'weather': [{'description': 'overcast clouds',
              'icon': '04n',
              'id': 804,
              'main': 'Clouds'}],
 'wind': {'deg': 90, 'speed': 3.1}}


<h2> Require data according to City entered

In [10]:
def getData(city):
    requestString=u'http://api.openweathermap.org/data/2.5/weather?q=%s&APPID=%s' % (city,a)
    
    res=requests.get(requestString)
    
    return res

In [11]:
res=getData('La paz')

In [12]:
res.status_code

200

<h2>Ingesting weather data into MongoDB

In [13]:
client=MongoClient('localhost')
db=client.packt

In [14]:
weatherCollection=db.weather

In [15]:
res.json()

{'coord': {'lon': -110.32, 'lat': 24.15},
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'scattered clouds',
   'icon': '03d'}],
 'base': 'stations',
 'main': {'temp': 299.5,
  'pressure': 1014,
  'humidity': 74,
  'temp_min': 299.15,
  'temp_max': 299.82},
 'visibility': 16093,
 'wind': {'speed': 1.5, 'deg': 260},
 'clouds': {'all': 40},
 'dt': 1574183899,
 'sys': {'type': 1,
  'id': 7137,
  'country': 'MX',
  'sunrise': 1574170762,
  'sunset': 1574210055},
 'timezone': -25200,
 'id': 4000900,
 'name': 'La Paz',
 'cod': 200}

In [17]:
#success = weatherCollection.insert_one(res.json())
#success = weatherCollection.

In [18]:
success.acknowledged

True

<h2> Get list of cities

In [94]:
!wget http://bulk.openweathermap.org/sample/city.list.json.gz

--2019-10-21 17:37:11--  http://bulk.openweathermap.org/sample/city.list.json.gz
Resolving bulk.openweathermap.org (bulk.openweathermap.org)... 136.243.88.22
Connecting to bulk.openweathermap.org (bulk.openweathermap.org)|136.243.88.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4292641 (4,1M) [text/plain]
Saving to: ‘city.list.json.gz’

city.list.json.gz   100%[===================>]   4,09M  3,17MB/s    in 1,3s    

2019-10-21 17:37:13 (3,17 MB/s) - ‘city.list.json.gz’ saved [4292641/4292641]



In [98]:
!gunzip city.list.json.gz

In [99]:
!head city.list.json

[
  {
    "id": 707860,
    "name": "Hurzuf",
    "country": "UA",
    "coord": {
      "lon": 34.283333,
      "lat": 44.549999
    }
  },


In [19]:
with open('city.list.json','r') as inFile:
    citiesJson=json.loads(inFile.read())

In [20]:
citiesJson[0]

{'id': 707860,
 'name': 'Hurzuf',
 'country': 'UA',
 'coord': {'lon': 34.283333, 'lat': 44.549999}}

In [21]:
type(citiesJson)

list

In [22]:
len(citiesJson)

209579

<h2>Limit to CHilean cities

In [23]:
citiesJsonCL=list(filter(lambda x:x[u'country']==u'CL',citiesJson))

In [24]:
len(citiesJsonCL)

369

In [25]:
cities=list(map(lambda x:x['name'],citiesJsonCL))
ids=list(map(lambda x:x['id'],citiesJsonCL))

In [26]:
res=getData(cities[0])

In [27]:
res

<Response [200]>

<h2>Cycle through cities

In [95]:
for i, name in zip(ids,cities):
    
    res=getData(name) 
    
    if not res.status_code==200:
        print ('Error grabbing data for %s' %name)
        print (res.reason)            
    
    else:
        try:
            weatherCollection.insert_one(res.json()) 
        except e:
            print ('Error inserting into DB' % e)
            print ('city %s' % name)
            
    time.sleep(1)
    #sleep so we do not thrash the API

Error grabbing data for Longavi
Not Found
Error grabbing data for Machali
Not Found
Error grabbing data for Vicuna
Not Found
Error grabbing data for Villa Presidente Frei, Nunoa, Santiago, Chile
Not Found


In [97]:
weatherCollection.count()

/home/fermat/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


2689

In [98]:
weatherCollection=db.weatherCollection
#weatherCollection=db.weatherPast

In [99]:
weatherCollection.count()

/home/fermat/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


2689

In [100]:
def getTimestamp(dt):
    return pd.datetime.fromtimestamp(dt)

In [101]:
def getDate(tstamp):
    dt=datetime.datetime.fromtimestamp(tstamp)
    return dt.strftime('%b %d - %H:%m')

<h2>Querying MongoDB for useful information

In [1]:
cur=weatherCollection.find()

NameError: name 'weatherCollection' is not defined

In [103]:
sortedCur=cur.sort('main.temp')

In [106]:
for doc in sortedCur.limit(10):
    try:
        print (doc['name'])
        print (doc['main']['temp'])
    except:
        print ('Error: missing name/temp')

Las Animas
263.65
Las Animas
265.5
Las Animas
265.88
Las Animas
265.88
Las Animas
265.98
Las Animas
265.98
Las Animas
266
Las Animas
266
Las Animas
266.08
Las Animas
269.17


In [110]:
cur.count()

/home/fermat/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


2689

<h2>Find the max recorded temperatures over time per station

In [117]:
pipeline=[]
pipeline.append({'$group':{'_id':'$name','maxTemp':{'$max':'$main.temp'}}})
#Group measurements by city name, extract maximum recorded temp for each
pipeline.append({'$limit':10})
#Limit results to first 10 cities

In [118]:
cur = weatherCollection.aggregate(pipeline=pipeline)

In [121]:
for d in cur:
    print (d['_id'],d['maxTemp'])

Barrio Bellavista 304.33
Provincia de Santiago 302.84
Ostrov Paskhi 295.15
Las Gaviotas 299.15
Santiago 296.63
Palena 287.15
Balmaceda 284.73
Chucumata 292.15
Santa Teresa de Lo Ovalle 299.24


<h2>Get datetime of max temp per station

In [122]:
pipeline=[]
pipeline.append({'$match':{'name':{'$exists':True}}})
#Filter out dirty rows
pipeline.append({'$sort':{'name':1,'main.temp':-1}})
#Sort by name (esc) and temperature (asc)
pipeline.append({'$group':{'_id':'$name','maxTemp':{'$first':'$main.temp'},'date':{'$first':'$dt'}}})
#Group by name, grab maximum temperature and date of temperature
#Sorted by temp so grab first document from each group
pipeline.append({'$limit':10})
#limit to first 10 cities

In [128]:
cur=weatherCollection.aggregate(pipeline=pipeline)

In [129]:
for d in cur:
    print (getDate(d['date']),d['_id'],d['maxTemp'])
    print ('')

Oct 30 - 22:10 Villa Paulina 299.23

Oct 31 - 00:10 Villa Alhué 297.15

Oct 30 - 22:10 Villa Alemana 298.02

Oct 30 - 22:10 Villa Alegre 293.5

Oct 30 - 14:10 Victoria 301.37

Oct 30 - 22:10 Vichuquén 291.02

Oct 30 - 22:10 Tongoy 290.15

Oct 30 - 22:10 Tongoicillo 290.15

Oct 30 - 14:10 Tome 284.15

Oct 30 - 22:10 Teno 296.15



<h2>Do sanity check for one city

In [131]:
for d in weatherCollection.find({'name':'Villa Paulina'}):
    print (getDate(d['dt']),'',d['main']['temp'])

Oct 30 - 14:10  288.52
Oct 30 - 22:10  299.23
Oct 30 - 22:10  299.23
Oct 30 - 22:10  299.15
Oct 30 - 22:10  299.15
Oct 30 - 23:10  298.66
Oct 30 - 23:10  298.66
Oct 30 - 23:10  298.11
Oct 30 - 23:10  298.62
